In [1]:
import numpy as np 
import random 
import re 
import tensorflow as tf 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 

learning_rate = 1e-2 #学习率设置为0.01

In [2]:
positive_training_data = "positive_training_dataset.txt"
negative_training_data = "negative_training_dataset.txt" 
positive_testing_data = "positive_testing_dataset.txt" 
negative_testing_data = "negative_testing_dataset.txt"#数据集的路径

In [3]:
def RNA_to_feature(str_rna, str_structure): 
#此函数将输入的RNA及其二级结构转化为输入的特征值 
    # 接受两个字符串,第一个字符串是基因序列,第二个字符串是对应的二级结构 
#     print(str_rna,str_structure)
#     energy = re.findall(r'-{0,1}\d{1,2}.\d{1,2}', str_structure)[0] 
    
    #利用正则表达式提取自由能 
    rna, structure = '','' 
    
    #初始化RNA和二级结构的数组 
    for i in range(len(str_rna)): 
        #这个循环用来保证二级结构和编码同样长度，同时避免混入非RNA编码 
        if str_rna[i] is not 'A' and str_rna[i] is not 'C' and str_rna[i] is not 'G' and str_rna[i] is not 'U': 
            continue 
        rna += str_rna[i] 
        structure += str_structure[i] 
        structure2int = [0 for i in range(len(structure))] 
        for i in range(len(structure)): 
            
        #这个循环将二级结构的编码转换为0，1编码，方便三元组编码的运用 
            if structure[i] is '(' or ')': 
                structure2int[i]= 1 
            if structure[i] is '.': 
                structure2int[i]= 0 
    #接下来构造一个32维特征，用于储存三元组编码方式的特征值，以字典形式给出。并初始化为0。
    rna_dic = {'A000': 0, 'A001': 0, 'A010': 0, 'A011': 0, 'A100': 0,'A101': 0, 'A110': 0, 'A111': 0, 
               'C000': 0, 'C001': 0, 'C010': 0, 'C011': 0, 'C100': 0,'C101': 0, 'C110': 0, 'C111': 0, 
               'G000': 0, 'G001': 0, 'G010': 0, 'G011': 0, 'G100': 0,'G101': 0, 'G110': 0, 'G111': 0, 
               'U000': 0, 'U001': 0, 'U010': 0, 'U011': 0, 'U100': 0,'U101': 0, 'U110': 0, 'U111': 0} 
    
    #然后搭建一个特征数组，方便判断使用。 
    rna_index = ['A000', 'A001', 'A010', 'A011', 'A100','A101', 'A110', 'A111',
                 'C000', 'C001', 'C010', 'C011', 'C100','C101', 'C110', 'C111',
                 'G000', 'G001', 'G010', 'G011', 'G100','G101', 'G110', 'G111',
                 'U000', 'U001', 'U010', 'U011', 'U100','U101', 'U110', 'U111'] 
    
    for i in range(len(rna)-2): 
        str2all = rna[i]+ __builtins__.str(structure2int[i])+ __builtins__.str(structure2int[i+1])+ __builtins__.str(structure2int[i+2]) 
        rna_dic[str2all] += 1 
    formalInput = [0 for i in range(32)] 
    #初始化最终特征向量formalInput，作为最后返回的特征向量。
    count = 0; 
    for i in rna_index: 
        formalInput[count] = rna_dic[i] 
        count += 1
#     print(energy)
#     formalInput[32] = float(energy) 
    #print(formalInput)
    #最后，将formalInput作为最终特征向量返回。 
    return formalInput
    
    
    
    

In [4]:
def input_form_x(data_path): 
#数据读取函数 
    with open(data_path) as data: 
        #打开文件 
        
        form_x = [] 
        fp = data.readlines() 
        print(len(fp))
        for i in range(0, len(fp), 3): 
            #每次跳两个读取 
            
            form_x.append(RNA_to_feature(fp[i + 1], fp[i + 2])) 
            #将奇数行（RNA编码）和偶数行（二级结构）一同输入，一起进行转换
            
    return form_x



In [5]:



def data_process(): 
    X_train_correct = input_form_x(positive_training_data) 
    # 获取数据 
    Y_train_correct = [] 
    for i in range(len(X_train_correct)): 
        Y_train_correct.append([1,0]) 
        
    X_train_wrong = input_form_x(negative_training_data) 
    Y_train_wrong = [] 
    for i in range(len(X_train_wrong)):
        Y_train_wrong.append([0,1])
    X_test_correct = input_form_x(positive_testing_data) # 获取数据 
    Y_test_correct = [] 
    for i in range(len(X_test_correct)): 
        Y_test_correct.append([1,0]) 
    X_test_wrong = input_form_x(negative_testing_data) 
    Y_test_wrong = [] 
    for i in range(len(X_test_wrong)): 
        Y_test_wrong.append([0,1])#采取独热编码进行二分类，是 与 不是。 
    return X_train_correct,Y_train_correct,X_train_wrong ,Y_train_wrong ,X_test_correct ,Y_test_correct ,X_test_wrong ,Y_test_wrong
    #然后将所有数据返回。
     

In [7]:
x = tf.placeholder(tf.float32, [None, 32]) 
y = tf.placeholder(tf.float32, [None, 2])



def multilayer_perceptron(x, weights, biases): 
    # 隐藏层采用ReLU激活函数 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) 
    layer_1 = tf.nn.relu(layer_1) 
    # 采用ReLU激活函数 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) 
    layer_2 = tf.nn.relu(layer_2) 
    # 输出层采用线性激活函数 
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] 
    return out_layer 
    #返回输出层的值

    
weights = { 'h1': tf.Variable(tf.random_normal([32, 10])), 
           'h2': tf.Variable(tf.random_normal([10, 20])), 
           'out': tf.Variable(tf.random_normal([20, 2])) } 

biases = { 'b1': tf.Variable(tf.random_normal([10])), 
          'b2': tf.Variable(tf.random_normal([20])), 
          'out': tf.Variable(tf.random_normal([2])) } 

pred = multilayer_perceptron(x, weights, biases) 

logits = tf.nn.softmax(pred) 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)   



correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
#得出通过正确个数除以总数得出准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


    
init = tf.global_variables_initializer()
X_train_correct, Y_train_correct, X_train_wrong, Y_train_wrong, X_test_correct, Y_test_correct ,X_test_wrong, Y_test_wrong = data_process() 
#x_train, x_test, y_train, y_test = train_test_split(x_train + x_test, y_train + y_test, test_size=0.15,random_state=42) 
x_train, x_test, y_train, y_test = train_test_split(X_train_correct + X_train_wrong + X_test_correct + X_test_wrong,
                                                    Y_train_correct + Y_train_wrong + Y_test_correct + Y_test_wrong,
                                                    test_size=0.15,random_state=42) 
# 训练集和测试集的随机分割


with tf.Session() as sess: 
    sess.run(init) 
    #初始化参数 
    # Training cycle 
    for epoch in range(8000): 
        batch_index = random.randint(0, 1000) 
        #每一次从0-1000随机选出一个数，作为起始点 
        batch_data = x_train[batch_index:batch_index + 32] 
        batch_labels = y_train[batch_index:batch_index + 32] 
        #在上面所选的起始点开始，拿出32组数据进行一次训练 
        sess.run([optimizer, cost], feed_dict={x: batch_data, y: batch_labels})
        #进行训练 
        if epoch%300 == 0: 
            print(epoch)
            #accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
            print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test}))
            print("Loss:", sess.run(cost, {x:x_test,y: y_test}))
            
            #每隔300次 输出对应的次数以及测试集所测试的准确度 
    # Test model 
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
    # Calculate accuracy 
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test})) 
    #最后，当训练结束时，输出最终的准确率。

1452
1452
372
372
0
Accuracy: 0.5027322
Loss: 50.620277
300
Accuracy: 0.8360656
Loss: 0.6726852
600
Accuracy: 0.86885244
Loss: 0.43444473
900
Accuracy: 0.8797814
Loss: 0.5457429
1200
Accuracy: 0.8579235
Loss: 0.5302412
1500
Accuracy: 0.86885244
Loss: 0.47272348
1800
Accuracy: 0.863388
Loss: 0.5160681
2100
Accuracy: 0.87431693
Loss: 0.62030965
2400
Accuracy: 0.8579235
Loss: 0.64787257
2700
Accuracy: 0.852459
Loss: 0.65693796
3000
Accuracy: 0.87431693
Loss: 0.70959437
3300
Accuracy: 0.8306011
Loss: 0.80710167
3600
Accuracy: 0.852459
Loss: 0.6136467
3900
Accuracy: 0.863388
Loss: 0.78176916
4200
Accuracy: 0.8797814
Loss: 0.6506654
4500
Accuracy: 0.8797814
Loss: 0.78422165
4800
Accuracy: 0.84153
Loss: 1.0122429
5100
Accuracy: 0.86885244
Loss: 0.8042573
5400
Accuracy: 0.8579235
Loss: 0.868407
5700
Accuracy: 0.87431693
Loss: 0.9550775
6000
Accuracy: 0.86885244
Loss: 0.59328663
6300
Accuracy: 0.89617485
Loss: 1.241212
6600
Accuracy: 0.89071035
Loss: 0.9411601
6900
Accuracy: 0.87431693
Loss: 1.